In [1]:
# Python Script to Extract tweets of a 
# particular Hashtag using Tweepy and Pandas

# import modules
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from collections import defaultdict
import datetime
import tweepy
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\axbae\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\axbae\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# function to display data of each tweet
def printtweetdata(n, ith_tweet):
    print()
    print(f"Tweet {n}:")
    print(f"Username:{ith_tweet[0]}")
    print(f"Description:{ith_tweet[1]}")
    print(f"Location:{ith_tweet[2]}")
    print(f"Following Count:{ith_tweet[3]}")
    print(f"Follower Count:{ith_tweet[4]}")
    print(f"Total Tweets:{ith_tweet[5]}")
    print(f"Retweet Count:{ith_tweet[6]}")
    print(f"Tweet Text:{ith_tweet[7]}")
    print(f"Hashtags Used:{ith_tweet[8]}")
    print(f"Date:{ith_tweet[9]}")

In [3]:
#importer dansk som sprog
#pip install afinn
from afinn import Afinn
afinn = Afinn(language='da')
afinn.score('Hvis ikke det er det mest afskyelige flueknepperi...')
-6.0

#user = api.get_status(id)
  
# fetching the created_at attribute
#created_at = status.created_at 

#today = datetime.date.today()

-6.0

In [141]:
# function to perform data extraction
def scrape(words, date_since, numtweet):
      
    # Creating DataFrame using pandas
    db = pd.DataFrame(columns=['username', 'description', 'location', 'following',
                               'followers', 'totaltweets', 'retweetcount', 'text', 'hashtags', 'date'])
      
    # We are using .Cursor() to search through twitter for the required tweets.
    # The number of tweets can be restricted using .items(number of tweets)
    tweets = tweepy.Cursor(api.search, q=words, lang="da",
                           since=date_since, tweet_mode='extended').items(numtweet)
     
    # .Cursor() returns an iterable object. Each item in 
    # the iterator has various attributes that you can access to 
    # get information about each tweet
    list_tweets = [tweet for tweet in tweets]
      
    # Counter to maintain Tweet Count
    i = 1  
      
    # we will iterate over each tweet in the list for extracting information about each tweet
    for tweet in list_tweets:
        username = tweet.user.screen_name
        description = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']
        date = tweet.created_at
          
        # Retweets can be distinguished by a retweeted_status attribute,
        # in case it is an invalid reference, except block will be executed
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:
            text = tweet.full_text
        hashtext = list()
        for j in range(0, len(hashtags)):
            hashtext.append(hashtags[j]['text'])
          
        # Here we are appending all the extracted information in the DataFrame
        ith_tweet = [username, description, location, following,
                     followers, totaltweets, retweetcount, text, hashtext, date]
        db.loc[len(db)] = ith_tweet
        
          
        # Function call to print tweet data on screen
        printtweetdata(i, ith_tweet)
        i = i+1
    filename = 'dkpol.csv'
      
    # we will save our database as a CSV file.
    db.to_csv(filename)
  
  
if __name__ == '__main__':
      
    # Enter your own credentials obtained 
    # from your developer account
    consumer_key = "HlgemqHuJ83L0NkQbg4rZQLQ1"
    consumer_secret = "iT2MPE5T2n4pGqDlTbFsStnlPyRPgOvk64bEhdRA7ZTE86G60g"
    access_key = "1427158064092876800-MNinx9VxTMhQf07xIutt2upLZvAaPn"
    access_secret = "Utlu32drsCwKj3SnCAYU2qKZ66VNovYBxLxdRu4DeDWrA"
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
      
    # Enter Hashtag and initial date
    print("Enter Twitter HashTag to search for")
    words = input()
    print("Enter Date since The Tweets are required in yyyy-mm--dd")
    date_since = input()
      
    # number of tweets you want to extract in one run
    numtweet = 5000
    scrape(words, date_since, numtweet)
    print('Scraping has completed!')

Enter Twitter HashTag to search for
dkpol
Enter Date since The Tweets are required in yyyy-mm--dd
2021-08-01


TweepError: Twitter error response: status code = 429

### Combining dataframes

In [142]:
df1=pd.read_csv('df1.csv')
df2=pd.read_csv('df2.csv')
df3=pd.read_csv('df3.csv')

obs = [df1, df2, df3]

data_set = pd.concat(obs)
data_set = data_set.reset_index(drop = True)

data_set.to_csv('twitter_data.csv', encoding = 'utf-8', index= False)

#data_set

### Cleaning the DataFrame

In [6]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [109]:
#Dropping collumns
data_set1 = data_set.drop(['Unnamed: 0','username', 'description', 'followers', 'following'], axis=1).reset_index(drop = True)

#Creating collumn for hashtags
data_set1['hashtags'] = data_set1['text'].apply(lambda x: re.findall(r'#(\w+)', x))

#Calling text column
tweets = data_set1['text'].values

In [138]:
import numpy as np

def clean_tweet(tweet):
    #if type(tweet) == np.float64:
        #return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) 
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('[-.,*:]',' ', temp)
    temp = re.sub('\v', ' ', temp)
    temp = re.sub("[\\n]"," ", temp)
    return temp

clean_data = [clean_tweet(tweets) for tweets in tweets]

In [69]:
#Taking out emojis
sent = emoji_pattern.sub(r'', str(clean_data))

#Tokenizing the data 
sent = nltk.tokenize.word_tokenize(str(clean_data))

#Remove danish stopWords
stopWords = set(stopwords.words('danish'))
wordsFiltered = []

for w in sent:
    if w not in stopWords:
        wordsFiltered.append(w)

In [104]:
from pandas import DataFrame

In [114]:
# has only been lower-cased and tokenized (a few cells above)
d = defaultdict(int)
for i in wordsFiltered:
    for w in i:
        d[w]+=1
V = d.keys()
print("Vocabulary size:",len(V))

# inspect the 10 most frequent tokens:
{k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)[:10]}

Vocabulary size: 177
